### --- [Day 5: Binary Boarding](https://adventofcode.com/2020/day/5) ---

You board your plane only to discover a new problem: you dropped your boarding pass! You aren't sure which seat is yours, and all of the flight attendants are busy with the flood of people that suddenly made it through passport control.

You write a quick program to use your phone's camera to scan all of the nearby boarding passes (your puzzle input); perhaps you can find your seat through process of elimination.

Instead of zones or groups, this airline uses binary space partitioning to seat people. A seat might be specified like FBFBBFFRLR, where F means "front", B means "back", L means "left", and R means "right".

The first 7 characters will either be F or B; these specify exactly one of the 128 rows on the plane (numbered 0 through 127). Each letter tells you which half of a region the given seat is in. Start with the whole list of rows; the first letter indicates whether the seat is in the front (0 through 63) or the back (64 through 127). The next letter indicates which half of that region the seat is in, and so on until you're left with exactly one row.

For example, consider just the first seven characters of FBFBBFFRLR:

    Start by considering the whole range, rows 0 through 127.
    F means to take the lower half, keeping rows 0 through 63.
    B means to take the upper half, keeping rows 32 through 63.
    F means to take the lower half, keeping rows 32 through 47.
    B means to take the upper half, keeping rows 40 through 47.
    B keeps rows 44 through 47.
    F keeps rows 44 through 45.
    The final F keeps the lower of the two, row 44.

The last three characters will be either L or R; these specify exactly one of the 8 columns of seats on the plane (numbered 0 through 7). The same process as above proceeds again, this time with only three steps. L means to keep the lower half, while R means to keep the upper half.

For example, consider just the last 3 characters of FBFBBFFRLR:

    Start by considering the whole range, columns 0 through 7.
    R means to take the upper half, keeping columns 4 through 7.
    L means to take the lower half, keeping columns 4 through 5.
    The final R keeps the upper of the two, column 5.

So, decoding FBFBBFFRLR reveals that it is the seat at row 44, column 5.

Every seat also has a unique seat ID: multiply the row by 8, then add the column. In this example, the seat has ID 44 * 8 + 5 = 357.

Here are some other boarding passes:

    BFFFBBFRRR: row 70, column 7, seat ID 567.
    FFFBBBFRRR: row 14, column 7, seat ID 119.
    BBFFBBFRLL: row 102, column 4, seat ID 820.

As a sanity check, look through your list of boarding passes. What is the highest seat ID on a boarding pass?


In [1]:
import csv
import math

INPUT_FILE = 'input_d5.txt'
NUM_ROWS = 128

def str_to_num(s):
    n = 0
    for c in s:
        n <<= 1
        if c == 'B' or c == 'R':
            n += 1
    return n

def str_to_seat(s):
    row = str_to_num(s[:7])
    col = str_to_num(s[7:])
    return row, col

def get_seat_id(row, col):
    return 8 * row + col

def scan_str(s):
    row, col = str_to_seat(s)
    return get_seat_id(row, col)

def max_seat_id(input_file):
    max_id = 0
    with open(input_file) as f:
        reader = csv.reader(f)
        for row in reader:
            max_id = max(max_id, scan_str(row[0]))
    return max_id
            

In [2]:
# Test the examples
row = str_to_num('FBFBBFF')
col = str_to_num('RLR')
assert row == 44
assert col == 5

assert scan_str('FBFBBFFRLR') == 357
assert scan_str('BFFFBBFRRR') == 567
assert scan_str('FFFBBBFRRR') == 119
assert scan_str('BBFFBBFRLL') == 820

In [3]:
# Part 1 solution ####################################
print(max_seat_id(INPUT_FILE))

878


##### Part 2
Ding! The "fasten seat belt" signs have turned on. Time to find your seat.

It's a completely full flight, so your seat should be the only missing boarding pass in your list. However, there's a catch: some of the seats at the very front and back of the plane don't exist on this aircraft, so they'll be missing from your list as well.

Your seat wasn't at the very front or back, though; the seats with IDs +1 and -1 from yours will be in your list.

In [4]:
def find_empty_seat(input_file, verbose=False):
    # Store each row as a bit array of seats, stored in col order 76543210
    seat_rows = [0] * NUM_ROWS
    min_seat_id = 1024
    max_seat_id = 0
    
    with open(input_file) as f:
        reader = csv.reader(f)
        for row in reader:
            row, col = str_to_seat(row[0])
            this_seat_id = get_seat_id(row, col)
            min_seat_id = min(min_seat_id, this_seat_id)
            max_seat_id = max(max_seat_id, this_seat_id)
                
            # Store the column encountered as 1 in the bit array
            seat_rows[row] = (1 << col) | seat_rows[row]
    
    if verbose:
        print(f'min seat encountered was {min_seat_id}, max was {max_seat_id}')
    
    for row_num in range(NUM_ROWS):
        # Look for rows with at least 1 seat filled, but not all 8
        if seat_rows[row_num] ^ 255 > 0 and seat_rows[row_num] > 0:
            # Find the highest column number of an empty seat in this row
            # i.e., the most significant bit after doing row XOR 255
            col_num = math.floor(math.log(seat_rows[row_num] ^ 255, 2))
            empty_seat_id = get_seat_id(row_num, col_num)
            if verbose:
                print(f'row {row_num} contains empty seat in col {col_num}: {seat_rows[row_num]:b} (ID:{get_seat_id(row_num, col_num)})')

            # Ignore empty seat numbers outside the range of IDs encountered (they don't exist)
            if empty_seat_id > min_seat_id and empty_seat_id < max_seat_id:
                return empty_seat_id
            

In [5]:
# Part 2 solution #############################################
find_empty_seat(INPUT_FILE, True)

min seat encountered was 54, max was 878
row 6 contains empty seat in col 5: 11000000 (ID:53)
row 63 contains empty seat in col 0: 11111110 (ID:504)


504